# Ngày 5: Shared Variable và Spark UI

#### I. Shard Variable
Spark theo mặc định không hỗ trợ thay đổi trạng thái biến được chia sẻ trên các Worker Node. Share variables giúp giải quyết một số giới hạn của kiến trúc này.

**1. Broadcast Variable**
   
*Khái niệm*: Broadcast variables dùng để chia sẻ một biến chỉ đọc đến tất cả worker node, nhằm giảm việc truyền dữ liệu lặp đi lặp lại.

*Sử dụng:* 
- Broadcast một từ điển lớn hoặc dữ liệu tham khỏa để tất cả worker node có thể truy cập mà không phải tải lại từ đầu.
- Lợi ích của Broadcast variables là giảm băng thông và cải thiện hiệu suất.

*Cú pháp và ví dụ:*
```python
    broadcast_val = sc.broadcast(large_dict)
    rdd.map(lambda x : some_function(x , broadcast_var.value))
```

![BroadCast](image_1/broadcast_2.png)

**2. Accumulators:**

*Khái niệm:* Accumulators là các biến có thể đếm hoặc tổng hợp dữ liệu trên các worker node, thường dùng để theo dõi số liệu hoặc lỗi mà không ảnh hưởng đến trạng thái ứng dụng.

*Sử dụng:* Theo dõi số lượng bản ghi lỗi hoặc số liệu thống kê

*Cú pháp và ví dụ:* 

```python
    accumulator = sc.accumulator(0)
    rdd.foreach( lambda x: accumulator.add(1) if x == 'error' else None)
```

![Accumulator](image_1/accumulator_2.png)


### II. Spark UI

Ba thành phần chính của Spark UI: **JOB  -  STAGE - TASK**

- Một JOB có nhiều STAGE
- Một STAGE có nhiều TASK

![Spark UI](image_1/SparkUI.png)

<p style = "color:green;">Vậy JOB, STAGE, TASK được hình thành như thế nào ?</p>

1. Sự hình thành JOB

Bản chất 1 JOB trong spark UI là 1 Actions khi ta thực hiện một Application trên RDD.

1 JOB = 1 ACTION

2. Sự hình thành của STAGE

- Một JOB bao gồm nhiều Stage, đại diện cho các bước trong quá trình thực thi.
- Ở đây ta lưu ý với hiện tượng Shuffle, Wide transformation có suffle còn narrow transformation thì không.
- Một Job có nhiều Stage với:
  - Số lượng Stage = Số lượng Wide Transformations + 1

3. Sự hình thành TASK

Số lượng **TASK** = Số lượng **PARTITION**

_Vậy câu hỏi được đặt ra là, làm thế nào để biết số lượng PARTITION mà SPARK sử lí._

Trước tiên phải hiểu về Task và Partition trong kiến trúc Cluster của Spark

![executor](image_1/executor.png)

Các **task** bên trong các **Executor** là các **Partition**



_Vậy SPARK chia partition như thế nào ?_

Về kiến thức, việc chia ra các **Executor và Partition** thuộc phạm vi **tối ưu** khi làm việc với Spark

- Ở Executor: thường người ta chia ra trong 1 Executor sẽ có 5 Core CPU, mỗi Core CPU sẽ đảm nhiệm 1 task và thực hiện sử lí song song các task đó trong cùng 1 Executor

- Ở Partiton(task):
    - Mặc định: Spark có cơ chế tự động phân tách Partition và tự động tối ưu số lượng Partion
        ```python
            from pyspark.sql import SparkSesstion
            spark = SparkSession.\
                builder.\
                master("local[4]").\
                appName("nb_partition").\
                getOrCreate()   

            orders_schema = "order_id long, order_date date, customer_id long, order_status string"

            orders_df = spark.read \
                .format("csv") \
                .schema(orders_schema) \
                .load("C:/data/orders_1gb.csv")

            spark.conf.get("spark.sql.shuffle.partitions")
            # Kết quả mặc định là 200 -> Tức là Mặc định khi load dữ liệu vào Spark sẽ chia 200 partition

            spark.conf.get("spark.sql.adaptive.enabled")
            # Tuy nhiên, khi xem spark UI sẽ thấy số lượng Partiton nhỏ hơn rất nhiều
            # Nguyên nhân là Spark có cơ chế tối ưu số lượng Partion

            spark.conf.set("spark.sql.adaptive.enabled", "false")
            # Tắt tối ưu sẽ thấy số lượng partion là 200 trong Spark UI

        ```
    - Tuy nhiên, là một Data Engineer, bạn không thể phụ thuộc hoàn toàn vào Spark, tùy dự án cụ thể mà 
bạn là người data engineer phải tính ra số lượng Partion phù hợp, số executor phù hợp với dự án.

    - Theo kinh nghiệm, ta có thể tính số lượng Partition theo công thức sau:
        - Nếu 1 File : SL Partition = MAX[ (File_Size)/128MB ,  Số lượng Core]
        - Nếu nhiều File: SL Partition = MAX [Số lượng File/(128/file_size_average  +  4MB)  ,  Số lượng Core]

![SL Partion](image_1/partition_2.png)